# 

# Time dependent arrivals via thinning

## Imports

In [1]:

from typing import Optional


## `sim-tools` imports

In [2]:
from sim_tools.datasets import load_banks_et_al_nspp

## Load example data from Banks et al.

In [3]:
data = load_banks_et_al_nspp()
data

,t,mean_iat,arrival_rate
0,0,15,0.066667
1,60,12,0.083333
2,120,7,0.142857
3,180,5,0.200000
4,240,8,0.125000
5,300,10,0.100000
6,360,15,0.066667
7,420,20,0.050000
8,480,20,0.050000


In [4]:
import itertools
import pandas as pd
import numpy as np

class NSPPThinning:

    def __init__(self, data, random_seed1: Optional[int]=None, random_seed2: Optional[int]=None):
        '''
        Non Stationary Poisson Process via Thinning.

        Time dependency is handled for a single table
        consisting of equally spaced intervals.         

        Params:
        ------
        data: pandas.DataFrame
            list of time points during a period for transition between rates
            and list arrival rates in that period. Labels should be "t"
            and "arrival_rate" respectively.           

        random_seed1: int
        
        random_seed2: int
        '''

        self.data = data
        self.arr_rng = np.random.default_rng(random_seed1)
        self.thinning_rng = np.random.default_rng(random_seed2)
        self.lambda_max = data['arrival_rate'].max()
        # assumes all other intervals are equal in length.
        self.interval = int(data.iloc[1]['t'] - data.iloc[0]['t'])
        self.rejects_last_sample = None

    def sample(self, simulation_time: float) -> float:
        '''
        Run a single iteration of acceptance-rejection
        thinning alg to sample the next inter-arrival time

        Params:
        ------
        simulation_time: float
            The current simulation time.  This is used to look up
            the arrival rate for the time period.

        Returns:
        -------
        float
            The inter-arrival time
        '''

        for _ in itertools.count():

            # this gives us the index of dataframe to use
            t = int(simulation_time // self.interval) % len(self.data)
            lambda_t = self.data['arrival_rate'].iloc[t]
            
            # set to a large number so that at least 1 sample taken!
            u = np.Inf

            # included for audit and tracking purposes.
            self.rejects_last_sample = 0.0
            
            interarrival_time = 0.0
            
            # reject samples if u >= lambda_t / lambda_max
            while u >= (lambda_t / self.lambda_max):
                self.rejects_last_sample += 1
                interarrival_time += self.arr_rng.exponential(1/self.lambda_max)
                u = self.thinning_rng.uniform(0.0, 1.0)
            
            return interarrival_time

In [5]:
# create arrivals and set random number seeds
SEED_1 = 42
SEED_2 = 101
arrivals = NSPPThinning(data, SEED_1, SEED_2)

# number of arrivals to simulate
n_arrivals = 10

# run simulation
simulation_time = 0.0
for _ in range(n_arrivals):
    iat = arrivals.sample(simulation_time)
    simulation_time += iat
    print(f'{simulation_time:.2f} Patient arrival (IAT={iat:.2f})')


37.46 Patient arrival (IAT=37.46)
73.02 Patient arrival (IAT=35.56)
89.41 Patient arrival (IAT=16.39)
96.79 Patient arrival (IAT=7.38)
98.37 Patient arrival (IAT=1.58)
104.94 Patient arrival (IAT=6.57)
112.30 Patient arrival (IAT=7.35)
121.49 Patient arrival (IAT=9.19)
127.62 Patient arrival (IAT=6.14)
135.25 Patient arrival (IAT=7.63)
